## <center>Підготовка та аналіз даних</center>
## <center>Лабораторна робота №2</center>
#### <center>ФБ-25 Кравченко Максим</center>

Імпортуємо потрібні нам бібліотеки

In [2]:
import datetime
import requests
import urllib.request
import pandas as pd
import os


Скачуємо VHI індекси для всіх областей
Також додаємо в назву дату та час скачування

In [5]:
for ids in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={ids}&year1=1981&year2=2024&type=Mean"
    response = requests.get(url)
    if response.status_code == 200:
        if not os.path.exists('vhi'):
            os.mkdir('vhi')
            print(f'The folder is created')
        date_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        vhi_url = urllib.request.urlopen(url)
        file_name = f'vhi/vhi_id_{ids}_{date_now}.csv'
        out = open(file_name, 'wb')
        out.write(vhi_url.read())
        out.close()
        print(f"VHI from id {ids} was downloaded at {date_now}")
    else:
        print(f"Download complete")
        break

VHI from id 1 was downloaded at 2024-04-06_21-43-55
VHI from id 2 was downloaded at 2024-04-06_21-44-34
VHI from id 3 was downloaded at 2024-04-06_21-44-36
VHI from id 4 was downloaded at 2024-04-06_21-44-39
VHI from id 5 was downloaded at 2024-04-06_21-44-44
VHI from id 6 was downloaded at 2024-04-06_21-44-46
VHI from id 7 was downloaded at 2024-04-06_21-44-48
VHI from id 8 was downloaded at 2024-04-06_21-44-51
VHI from id 9 was downloaded at 2024-04-06_21-44-53
VHI from id 10 was downloaded at 2024-04-06_21-44-55
VHI from id 11 was downloaded at 2024-04-06_21-44-58
VHI from id 12 was downloaded at 2024-04-06_21-45-00
VHI from id 13 was downloaded at 2024-04-06_21-45-03
VHI from id 14 was downloaded at 2024-04-06_21-45-05
VHI from id 15 was downloaded at 2024-04-06_21-45-08
VHI from id 16 was downloaded at 2024-04-06_21-45-10
VHI from id 17 was downloaded at 2024-04-06_21-45-12
VHI from id 18 was downloaded at 2024-04-06_21-45-16
VHI from id 19 was downloaded at 2024-04-06_21-45-18
VH

Даємо імена стовбцям, зчитуємо файли у фрейм

In [6]:
folder = 'vhi'
files = os.listdir(folder)
# ids = 0
df_all = pd.DataFrame()

for file_name in files:
    # ids += 1
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{folder}/{file_name}', header=1, names=headers, skiprows=1)[:-1]
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df['area'] = int(file_name.split("_")[2])
    # print(df.head(), '\n')
    # df_all = pd.concat([df_all, df], ignore_index=True)
    df = df.drop(columns=['empty'])
    df_all = df_all.dropna(axis=1, how='all')
    df_all = pd.concat([df_all, df]).drop_duplicates().reset_index(drop=True)
    
# zm = df_all.isna().sum()
# print(zm)
print(df_all, '\n')

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    10
...     ...   ...    ...     ...    ...    ...    ...   ...
57937  2024   9.0  0.152  284.35  64.56   2.42  33.49     9
57938  2024  10.0  0.167  286.52  64.04   2.70  33.37     9
57939  2024  11.0  0.179  288.09  62.75   3.03  32.89     9
57940  2024  12.0  0.193  289.61  62.78   4.37  33.57     9
57941  2024  13.0  0.203  290.96  61.15   7.87  34.51     9

[57942 rows x 8 columns] 



Змінимо індекси областей

In [7]:
dict_areas = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 13: 10, 14: 11, 15: 12,
              16: 13, 17: 15, 18: 14, 19: 16, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
df_all["area"] = df_all["area"].replace(dict_areas)
df_all.to_csv(f'vhi/df_all.csv', index=False)

print(df_all)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    21
...     ...   ...    ...     ...    ...    ...    ...   ...
57937  2024   9.0  0.152  284.35  64.56   2.42  33.49    20
57938  2024  10.0  0.167  286.52  64.04   2.70  33.37    20
57939  2024  11.0  0.179  288.09  62.75   3.03  32.89    20
57940  2024  12.0  0.193  289.61  62.78   4.37  33.57    20
57941  2024  13.0  0.203  290.96  61.15   7.87  34.51    20

[57942 rows x 8 columns]


Ряд VHI для області за вказаний рік, пошук екстремумів (min та max);

In [8]:
def vhi(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI']

def vhi_min(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].min()

def vhi_max(area, year):
    return df_all[(df_all["area"] == area) & (df_all["Year"] == year)]['VHI'].max()

df_all['Year'] = df_all['Year'].astype(int)

In [9]:
vhi(6, 2005)

31190    47.42
31191    51.20
31192    53.68
31193    54.35
31194    54.66
31195    56.10
31196    56.81
31197    56.06
31198    56.27
31199    56.51
31200    56.42
31201    56.02
31202    54.84
31203    52.12
31204    49.52
31205    48.44
31206    53.62
31207    52.32
31208    52.21
31209    51.77
31210    53.50
31211    54.49
31212    54.25
31213    55.50
31214    57.29
31215    57.93
31216    58.44
31217    65.13
31218    59.35
31219    61.40
31220    63.51
31221    64.39
31222    66.13
31223    65.04
31224    60.86
31225    57.68
31226    56.59
31227    54.98
31228    53.23
31229    52.48
31230    55.20
31231    57.22
31232    57.66
31233    58.95
31234    56.50
31235    54.77
31236    54.48
31237    53.22
31238    50.36
31239    47.41
31240    45.59
Name: VHI, dtype: float64

In [14]:
vhi_min(6, 2005)


45.59

In [15]:
vhi_max(6, 2005)

66.13

Ряд VHI за вказаний діапазон років для вказаних областей;

In [16]:
def vhi_range(year_min, year_max, areas):
    if not isinstance(areas, list) or not areas:
        return print('Empty or not a list')
    return df_all[(df_all['Year'] >= year_min) & (df_all['Year'] <= year_max) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [17]:
vhi_range(2000, 2005, [5, 2, 3])

,Year,VHI,area
35241,2000,24.65,2
35242,2000,27.49,2
35243,2000,31.36,2
35244,2000,37.28,2
35245,2000,40.85,2
...,...,...,...
48404,2005,40.70,3
48405,2005,42.92,3
48406,2005,41.13,3
48407,2005,41.49,3


Роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей

In [18]:
def extreme_droughts(procent):
    df_drouhts = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    result = grouped[grouped > (25 * procent / 100)].reset_index()
    return result

In [19]:
extreme_droughts(5)

,Year,area
0,1986,2
1,1993,2
2,2000,6
3,2007,5


Аналогічно для помірних посух

In [20]:
def moderate_droughts(procent, min=15, max=40):
    df_drouhts = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    grouped = df_drouhts.groupby('Year')['area'].nunique()
    count_areas = df_all['area'].nunique()
    result = grouped[grouped > (count_areas * procent / 100)].reset_index()
    return result

In [21]:
moderate_droughts(20)

,Year,area
0,1982,25
1,1983,25
2,1984,25
3,1985,25
4,1986,25
5,1987,25
6,1988,25
7,1989,25
8,1990,25
9,1991,24
